In [ ]:
import pandas as pd
basic_stats = pd.io.parsers.read_table('./Datos/basic_stats000',sep='\t',names=['zipcode','date','category','merchant','card', 'payment', 'avg', 'max', 'min', 'std'])


In [ ]:
len(basic_stats)

In [52]:
from elasticsearch import Elasticsearch
import pandas as pd
from time import time
import json
 
root_path="./Datos"
raw_data_path=root_path+"/"
csv_filename=raw_data_path + "basic_stats000"
col_names =['zipcode','date','category','merchant','card', 'payment', 'avg', 'max', 'min', 'std']


index_name="bbva"
type_name="basic_stats"

t0=time()
 

 
df=pd.io.parsers.read_table(csv_filename ,sep='\t',names=col_names)

# init ElasticSearch
es = Elasticsearch()
 
# init index
try :
    es.indices.delete(index_name)
except :
    pass
 
es.indices.create(index=index_name, ignore=400)
 
# start bulk indexing 
print "now indexing %s..."%(csv_filename)
 
df["no_index"] = [x+1 for x in range(len(df))]
# Convert into json
tmp = df.to_json(orient = "records")
# Load each record into json format before bulk
df_json= json.loads(tmp)
print df_json[0]

# Bulk index
for doc in df_json:
    es.index(body=doc, index=index_name, doc_type=type_name, id=doc['no_index'])
 
print "done in %.3fs"%(time()-t0)

now indexing ./Datos/basic_stats000...
{u'category': u'es_barsandrestaurants', u'merchant': 76, u'min': 0.1, u'max': 600.0, u'zipcode': 8001, u'std': 47.2536063819, u'date': u'2014-07-01', u'avg': 27.28, u'no_index': 1, u'payment': 410, u'card': 405}
done in 71.115s


In [ ]:
print csvfile

In [53]:
df_json[:2]

[{u'avg': 27.28,
  u'card': 405,
  u'category': u'es_barsandrestaurants',
  u'date': u'2014-07-01',
  u'max': 600.0,
  u'merchant': 76,
  u'min': 0.1,
  u'no_index': 1,
  u'payment': 410,
  u'std': 47.2536063819,
  u'zipcode': 8001},
 {u'avg': 34.76,
  u'card': 73,
  u'category': u'es_contents',
  u'date': u'2014-07-01',
  u'max': 189.36,
  u'merchant': 15,
  u'min': 7.0,
  u'no_index': 2,
  u'payment': 76,
  u'std': 27.3307925802,
  u'zipcode': 8001}]

In [ ]:
from datetime import datetime
from elasticsearch import Elasticsearch

# by default we connect to localhost:9200
es = Elasticsearch()
# create an index in elasticsearch, ignore status code 400 (index already exists)
es.indices.create(index='test-index', ignore=400)
# datetimes will be serialized
es.index(index="test-index", doc_type="test-type", id=42, body={"any": "data", "timestamp": datetime.now()})
# but not deserialized

es.get(index="test-index", doc_type="test-type", id=42)['_source']


In [ ]:
es.indices.delete('test-index')

In [ ]:
es.index(index="test-index", doc_type="test-type", body={"any": "data", "timestamp": datetime.now()})